In [119]:
import csv
import matplotlib.pyplot as plt
import numpy as np

In [120]:
chess_results = []

f = open('Data/chess.txt', 'r')
for item in f.read():
    if item != '\n':
        chess_results.append(int(item))

data = np.array(chess_results) 

Emission_Matrix = np.array([[0.3,0.6,0.1],[0.5,0,0.5]])
Initial_Distribution = np.array([[1,0]])
Transition_Matrix = np.array([[0.5,0.5], [0.8, 0.2]])

In [121]:
print(data.shape)
print(Emission_Matrix.shape)
print(Initial_Distribution.shape)
print(Transition_Matrix.shape)

(1000,)
(2, 3)
(1, 2)
(2, 2)


In [122]:
print(np.sum(Transition_Matrix[0,:]))

1.0


In [123]:
def forward(V, A, T, initial_dist):
    alpha = np.zeros((V.shape[0], T.shape[0]))
    alpha[0,:] = initial_dist*A[:,V[0]]
    alpha[0,:] = alpha[0,:]/np.sum(alpha[0,:])
    
    for t in range(1, V.shape[0]):
        for j in range(T.shape[0]):
            alpha[t,j] = alpha[t-1]@T[:,j]*A[j,V[t]]
        
        alpha[t,:] = alpha[t,:]/np.sum(alpha[t,:])
    
    return alpha

In [124]:
def backward(V, A, T):
    beta = np.zeros((V.shape[0], T.shape[0]))
    beta[V.shape[0] - 1,:] = np.ones((T.shape[0]))
    beta[V.shape[0] - 1,:] = beta[V.shape[0] - 1,:]/np.sum(beta[V.shape[0] - 1,:])
    
    for t in range(V.shape[0] -2, -1, -1):
        for j in range(T.shape[0]):
            beta[t,j]=(beta[t+1]* A[:,V[t+1]])@T[j,:]
    
        beta[t,:] = beta[t,:]/np.sum(beta[t,:])
    return beta

In [125]:
def EM(V,A,T,initial_dist):
    
    top_term = np.zeros((T.shape[0]))
    updated_transition_matrix = np.zeros((T.shape[0],T.shape[0]))
    
    alpha = forward(V,A,T,initial_dist)
    beta = backward(V,A,T)
    
    ## Expection Step
    for i in range(V.shape[0]-1):
        
        ## compute the responsibility for each sample
        response = np.ones((T.shape[0],T.shape[0]))
        
        for t in range(T.shape[0]):
            for t_plus in range(T.shape[0]):
                response[t][t_plus] = alpha[i][t]*beta[i+1][t_plus]*A[t_plus][V[i+1]]*T[t][t_plus]
                
        ## normalise the responsibility for the sample
        response /= np.sum(np.sum(response))
        ## add the resposibilities (compute the top term of th Maximisation Step)
        top_term = np.add(top_term,response)  
    
    ## Compute the bottom term of the maximisation step
    bottom_term = np.zeros((T.shape[0]))
    bottom_term = np.sum(top_term,axis = 1)
    ## Maximisation step
    updated_transition_matrix[0,:] =  top_term[0,:]/bottom_term[0]
    updated_transition_matrix[1,:] =  top_term[1,:]/bottom_term[1]
                                                                 
    return updated_transition_matrix

In [126]:
def WaumBelch(V,A,T,initial_dist):
    for i in range(100):
        T = EM(V,A,T,initial_dist)
    return T

In [127]:
updated_transition_matrix = WaumBelch(data,Emission_Matrix,Transition_Matrix,Initial_Distribution)

In [128]:
print(np.sum(updated_transition_matrix[0,:]))
print(np.sum(updated_transition_matrix[1,:]))

1.0
1.0


In [129]:
updated_transition_matrix

array([[0.83258179, 0.16741821],
       [0.0723181 , 0.9276819 ]])